In [1]:
pip install ollama

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install langchain langchain_community chromadb unstructured

     ---------------------------------------- 0.0/981.5 kB ? eta -:--:--
     - ------------------------------------- 41.0/981.5 kB 2.0 MB/s eta 0:00:01
     ----- -------------------------------- 143.4/981.5 kB 1.7 MB/s eta 0:00:01
     --------- ---------------------------- 235.5/981.5 kB 1.8 MB/s eta 0:00:01
     ----------------- -------------------- 440.3/981.5 kB 2.8 MB/s eta 0:00:01
     ----------------- -------------------- 440.3/981.5 kB 2.8 MB/s eta 0:00:01
     ---------------------------- --------- 747.5/981.5 kB 2.8 MB/s eta 0:00:01
     ---------------------------- --------- 747.5/981.5 kB 2.8 MB/s eta 0:00:01
     -------------------------------------- 981.5/981.5 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached charset_normalizer-3.3.2-cp311-cp311-win_amd64.whl.metadata (34 kB)
  Using cached pypdf-4.2.0-py3-none-any.whl.metadata (7.4 kB)
   -------------------------------------

Ok, now lets do our RAG

In [2]:
import ollama

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [4]:
from langchain.document_loaders import DirectoryLoader

In [5]:
loader = DirectoryLoader('../',glob="**/*.md")
embeddingModel = 'mxbai-embed-large'
llmModel = 'llama3'

In [6]:
docs = loader.load()
print(len(docs))

128


We can skip the splitting etc, as the documents are already split

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
print(len(splits))

Now we use mistral to create the embeddings and store them in chroma

In [8]:
%%time
embeddings = OllamaEmbeddings(model=embeddingModel)
vectorstore = Chroma.from_documents(documents=docs, embedding=embeddings,persist_directory='./chromadb')

CPU times: total: 172 ms
Wall time: 4min 46s


In [9]:
retriever = vectorstore.as_retriever()

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def ollama_llm(question, context):
    formatted_prompt = f"Question: {question}\n\nContext: {context}"
    response = ollama.chat(model=llmModel, messages=[{'role': 'user', 'content': formatted_prompt}])
    return response['message']['content']


In [10]:
# Define the RAG chain
def rag_chain(question):
    retrieved_docs = retriever.invoke(question)
    formatted_context = format_docs(retrieved_docs)
    #print(retrieved_docs)
    return ollama_llm(question, formatted_context)

In [11]:
%%time
result = rag_chain("What address should I use for jaeger agent on snappcloud?")
print(result)

According to the documentation, the Jaeger agent address for SnappCloud is:

`jaeger-agent:<cluster_id>-<region>`



Replace `<cluster_id>` with either `okd4` or `snappgroup`, depending on your cluster ID. And replace `<region>` with either `Teh-1` (Asiatech datacenter) or leave it blank if you're using the `snappgroup` cluster.

For example, if you're using the `okd4` cluster and Teh-1 region:

`jaeger-agent:okd4-Teh-1`

If you're using the `snappgroup` cluster (which is likely to be your case), simply use:

`jaeger-agent:snappgroup`

Please note that this address might change, so it's always a good idea to check the official SnappCloud documentation for any updates.
CPU times: total: 0 ns
Wall time: 20 s


In [13]:
%%time
result = rag_chain("What are the external IPs for snappcloud that i need to whitelist?")
print(result)

According to the SnappCloud documentation, the external IPs that need to be whitelisted for snappcloud are:

* `image-registry.apps.private.{globals.clusterID}.teh-1.snappcloud.io`
* `image-registry.apps.private.{globals.clusterID}.teh-2.snappcloud.io` (if you're importing images from another region)

These IPs are used for image registry operations, such as importing and pulling images.
CPU times: total: 0 ns
Wall time: 19 s
